In [1]:
import pandas as pd
import numpy as np
import copy

In [2]:
# genes-pathways annotation

path = './KEGG_pathways/20230205_kegg_hsa.gmt'

files = open(path,encoding='utf-8')

files = files.readlines()

paways_genes_dict = {}
for i in files: 
    paways_genes_dict[i.split('\t')[0].split('_')[0]] = i.replace('\n','').split('\t')[2:] 


In [3]:
#mirna-pathways annotation
path = './KEGG_pathways/kegg_anano.txt'

files = open(path,encoding='utf-8')

files = files.readlines()

paways_mirna_dict = {}
for i in files:
     keys = i.split(',')[0].split('|')[1]
     values1 = i.split(',')[1:-1]
     values2 =  i.split(',')[-1].replace('\n','')
     values1.append(values2)
     values1 =list(set(values1)) 
     paways_mirna_dict[keys] = values1

In [4]:
union_kegg = list(set(paways_genes_dict.keys()).intersection(set(paways_mirna_dict.keys())))

In [5]:
paways_genes_dicts ={}
paways_mirna_dicts ={}

for i in union_kegg:
    paways_genes_dicts[i] = paways_genes_dict[i]
    
for i in union_kegg:
    paways_mirna_dicts[i] = paways_mirna_dict[i]    

In [6]:

genes_existed_pathway = []

mirna_existed_pathway = []

for index in paways_genes_dicts.keys():
    genes_existed_pathway = genes_existed_pathway+ list(paways_genes_dicts[index])
genes_existed_pathway = set(genes_existed_pathway)


for index in paways_mirna_dicts.keys():
    mirna_existed_pathway = mirna_existed_pathway+ list(paways_mirna_dicts[index])
mirna_existed_pathway = set(mirna_existed_pathway)



In [7]:
print(len(genes_existed_pathway))
print(len(mirna_existed_pathway))

7768
812


In [8]:
#Loading data
snv_data = pd.read_csv("./LIHC_data/snv_data.csv",index_col = 0)

miRNA_data = pd.read_csv("./LIHC_data/miRNA_data.csv",index_col = 0)

mRNA_data = pd.read_csv("./LIHC_data/mRNA_data.csv",index_col = 0)

example_case = pd.read_csv('./LIHC_data/response.csv',index_col=0)

In [9]:
print(snv_data.shape)
print(mRNA_data.shape)
print(miRNA_data.shape)

(354, 1000)
(354, 1000)
(354, 200)


In [10]:
union_gene_snv = list(snv_data.columns)
union_gene_miRNA = list(miRNA_data.columns)
union_gene_mRNA = list(mRNA_data.columns)

In [11]:
pathway_union = list(paways_genes_dicts.keys())
len(pathway_union)

238

In [12]:
mask_list = [union_gene_snv,union_gene_mRNA]

gene_pathway_bp_dfs = []


for i in range(len(mask_list)):
    pathways_genes = np.zeros((len(pathway_union), len(mask_list[i]))) 
    for p  in pathway_union:
        gs = paways_genes_dicts[p]
        g_inds = [mask_list[i].index(g) for g in gs if g in mask_list[i]]
        p_ind = pathway_union.index(p)
        pathways_genes[p_ind, g_inds] = 1
    gene_pathway_bp = pd.DataFrame(pathways_genes, index=pathway_union, columns=mask_list[i])
    
    gene_pathway_bp_dfs.append(gene_pathway_bp)
    

pathways_genes = np.zeros((len(pathway_union), len(union_gene_miRNA))) 
for p  in pathway_union:
    gs = paways_mirna_dicts[p]
    g_inds = [union_gene_miRNA.index(g) for g in gs if g in union_gene_miRNA]
    p_ind = pathway_union.index(p)
    pathways_genes[p_ind, g_inds] = 1
gene_pathway_bp = pd.DataFrame(pathways_genes, index=pathway_union, columns=union_gene_miRNA)

gene_pathway_bp_dfs.append(gene_pathway_bp)
    


In [20]:
# gene_pathway_bp_dfs

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.initializers import glorot_uniform, Initializer
from tensorflow.keras.layers import Input, Dense, Dropout, Embedding, GlobalAveragePooling1D,Layer
from tensorflow.keras import initializers,activations,regularizers
from tensorflow.keras.regularizers import Regularizer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.python.framework.ops import disable_eager_execution
 
tf.compat.v1.disable_eager_execution()

In [14]:
class Biological_module(Layer):
    def __init__(self, units, mapp=None, nonzero_ind=None, kernel_initializer='glorot_uniform', W_regularizer=None,
                 activation='tanh', use_bias=True,bias_initializer='zeros', bias_regularizer=None,
                 bias_constraint=None,**kwargs):
        
        self.units = units
        self.activation = activation
        self.mapp = mapp
        self.nonzero_ind = nonzero_ind
        self.use_bias = use_bias
        
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.kernel_regularizer = regularizers.get(W_regularizer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activation_fn = activations.get(activation)
        super(Biological_module, self).__init__(**kwargs)

        
    def build(self, input_shape):
        
        input_dim = input_shape[1]
   

        if not self.mapp is None:
            self.mapp = self.mapp.astype(np.float32)

   
        if self.nonzero_ind is None:
            nonzero_ind = np.array(np.nonzero(self.mapp)).T
            self.nonzero_ind = nonzero_ind

        self.kernel_shape = (input_dim, self.units)
        

        nonzero_count = self.nonzero_ind.shape[0]   


        self.kernel_vector = self.add_weight(name='kernel_vector',
                                             shape=(nonzero_count,),
                                             initializer=self.kernel_initializer,
                                             regularizer=self.kernel_regularizer,
                                             trainable=True)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer
                                        )
        else:
            self.bias = None

        super(Biological_module, self).build(input_shape)  
      

    def call(self, inputs):
        
        
        trans = tf.scatter_nd(tf.constant(self.nonzero_ind, tf.int32), self.kernel_vector,
                           tf.constant(list(self.kernel_shape)))
    
        output = K.dot(inputs, trans)
        
    
        if self.use_bias:
            output = K.bias_add(output, self.bias)
            
        if self.activation_fn is not None:
            output = self.activation_fn(output)

        return output

    def get_config(self):
        config = {
            'units': self.units,
            'activation': self.activation,
            'use_bias': self.use_bias,
            'nonzero_ind': np.array(self.nonzero_ind),
          
            'bias_initializer': initializers.serialize(self.bias_initializer),
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),

            'kernel_initializer': initializers.serialize(self.kernel_initializer),
            'W_regularizer': regularizers.serialize(self.kernel_regularizer),

        }
        base_config = super(Biological_module, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
      
        return (input_shape[0], self.units)

In [15]:
class Self_Attention(Layer):
 
    def __init__(self, output_dim,  W_regularizer=None,**kwargs):
        self.output_dim = output_dim
        self.kernel_regularizer = regularizers.get(W_regularizer)
        super(Self_Attention, self).__init__(**kwargs)
 
    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel',
                                      shape=(3,input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      regularizer=self.kernel_regularizer,
                                      trainable=True)
 
        super(Self_Attention, self).build(input_shape)  
 
    def call(self, x):
        WQ = K.dot(x, self.kernel[0])
        WK = K.dot(x, self.kernel[1])
        WV = K.dot(x, self.kernel[2])
 

        print("K.permute_dimensions(WK.shape",(K.permute_dimensions(WK,[1,0]).shape))
 
        QK =  K.dot(K.permute_dimensions(WK,[1,0]),WQ)
    
 
        QK = QK / (64**0.5)
 
        QK = K.softmax(QK)
 
        print("QK.shape",QK.shape)
 
        V = K.dot(WV,QK)
        
        print(V.shape)
 
        return V

    def get_config(self):
        config = {
          
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),


        }
        base_config = super(Self_Attention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
 
    def compute_output_shape(self, input_shape):
 
        return (input_shape[0],input_shape[1],self.output_dim)
 

In [17]:
def create_model(snv_data,mRNA_data,miRNA_data):
    
    S_inputs_snv = Input(shape=(snv_data.shape[1],), dtype='float32',name= 'snv_inputs')
    
    S_inputs_mRNA = Input(shape=(mRNA_data.shape[1],), dtype='float32',name= 'mRNA_inputs')
  
    S_inputs_miRNA = Input(shape=(miRNA_data.shape[1],), dtype='float32',name= 'miRNA_inputs')
    

    h0_snv = Biological_module(gene_pathway_bp_dfs[0].shape[0],mapp =gene_pathway_bp_dfs[0].values.T, name = 'h0_snv',W_regularizer=l2(0.001))(S_inputs_snv)

    
    h0_mRNA = Biological_module(gene_pathway_bp_dfs[1].shape[0],mapp =gene_pathway_bp_dfs[1].values.T, name = 'h0_mRNA',W_regularizer=l2(0.001))(S_inputs_mRNA)

    
    h0_miRNA = Biological_module(gene_pathway_bp_dfs[2].shape[0],mapp =gene_pathway_bp_dfs[2].values.T, name = 'h0_miRNA',W_regularizer=l2(0.001))(S_inputs_miRNA)


    atten1 = Self_Attention(64,W_regularizer=l2(0.001))(h0_snv)
    atten2 = Self_Attention(64,W_regularizer=l2(0.01))(h0_mRNA)
    atten3 = Self_Attention(64,W_regularizer=l2(0.01))(h0_miRNA)
    
    feature_tal = tf.keras.layers.concatenate([atten1,atten2,atten3])

    
    h4 = tf.keras.layers.Dense(32,activation='tanh')(feature_tal)
    
    h5 = tf.keras.layers.Dense(1,activation='sigmoid')(h4)
    

    model = Model(inputs=[S_inputs_snv,S_inputs_mRNA,S_inputs_miRNA], outputs=h5)

    model.summary()

    opt = tf.keras.optimizers.Adam(learning_rate = 0.0001,decay=0.0001) 
    model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=['acc'])
    return model

In [18]:
#Evaluation function
import sklearn
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
   

from sklearn.metrics import precision_recall_curve
def get_metrics(true_score,pre_score,pre_probe):
    
  
    fpr, tpr, thresholds = metrics.roc_curve(true_score, pre_probe, pos_label=1)
   
    auc = metrics.auc(fpr, tpr)
    
    aupr = average_precision_score(true_score, pre_probe)
    
    pre, rec, thresholds = precision_recall_curve(true_score, pre_probe)    
    auprc  = metrics.auc(rec, pre)
    
    
    accuracy = accuracy_score(true_score,pre_score)
    
    f1 = metrics.f1_score(true_score, pre_score)
    
    precision = metrics.precision_score(true_score,pre_score)
    
    recall = metrics.recall_score(true_score,pre_score)
    
     
    print( print(confusion_matrix(true_score,pre_score)))
    return precision,accuracy,recall,f1,auc,aupr,auprc


def evaluates(y_test, y_pred):
    
    auc = metrics.roc_auc_score(y_test,y_pred)
    
    aupr = average_precision_score(y_test, y_pred)
    
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred)    
    auprc  = metrics.auc(recall, precision)
    
    pp = [1 if index>=0.5  else 0 for index in  y_pred ]
    
    pre = metrics.precision_score(y_test,pp)
    
    f1 = metrics.f1_score(y_test,pp)
    
    rec = metrics.recall_score(y_test,pp)
    
    acc = metrics.accuracy_score(y_test,pp)
    
    print(confusion_matrix(y_test,pp))
    return pre,acc,rec,f1,auc,aupr,auprc

In [19]:
snv_data = snv_data.loc[example_case.index]

miRNA_data = miRNA_data.loc[example_case.index]

mRNA_data = mRNA_data.loc[example_case.index]

example_case = example_case.loc[example_case.index]

y = example_case['response'].values

In [20]:

n_samples =example_case['response'].values

print(len(n_samples),n_samples.sum(),(len(n_samples) -n_samples.sum()))

x_0 =  len(n_samples) / (2*  (len(n_samples) -n_samples.sum()))
x_1 =  len(n_samples) / (2*  n_samples.sum())

print(x_0,x_1)

354 171 183
0.9672131147540983 1.0350877192982457


In [31]:
#feature importance
from keras.models import Sequential
def get_layers(model, level=1):
    layers = []
    for i, l in enumerate(model.layers):

        if type(l) == Sequential:
            layers.extend(get_layers(l, level + 1))
        else:
            layers.append(l)

    return layers



positive_example= example_case[example_case['response']==1]
snv_data_p = snv_data.loc[positive_example.index].values
mRNA_data_p  = mRNA_data.loc[positive_example.index].values
miRNA_data_p = miRNA_data.loc[positive_example.index].values


data_pd_s = [snv_data,mRNA_data,miRNA_data]
data_pd_name =['snv','mRNA','miRNA']
def get_weigjts(model,p):
    
   
    feed_dict ={}
    model_layers = get_layers(model)
    multi_data_inputs = [snv_data_p,mRNA_data_p,miRNA_data_p]
    for i in range(0,3):
        feed_dict[model.get_layer(model_layers[i].name).output] = multi_data_inputs[i]
        
    for k in range(0,3):
        meth_inputs_name = model_layers[k].name
        X = model.get_layer(meth_inputs_name).output 
        Y  = model.outputs[-1]
        ret = [g * (x - b) for g, x, b in zip(tf.gradients(Y, X), [X], [np.zeros((multi_data_inputs[k].shape))[0].shape])]
        sess = tf.compat.v1.InteractiveSession()
        sess.run(tf.compat.v1.initialize_all_variables())
        evaluated_gradients = sess.run(ret,feed_dict=feed_dict)
        gene_pd = pd.DataFrame(columns=['genes','values'])
        gene_pd['genes'] = data_pd_s[k].columns
        gene_pd['values'] = evaluated_gradients[0].sum(axis=0)
        gene_pd.to_csv('./coef_weight/LIHC/h{}/{}.csv'.format(p,data_pd_name[k]))

In [32]:
#Five-fold cross validation
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=1029) 

kfscore = []
p = 0
for train_index, test_index in skf.split(snv_data.values,y):

    snv_train_x = snv_data.values[train_index]
    snv_test_x  = snv_data.values[test_index]

    mRNA_train_x = mRNA_data.values[train_index]
    mRNA_test_x  = mRNA_data.values[test_index]

    miRNA_train_x = miRNA_data.values[train_index]
    miRNA_test_x  = miRNA_data.values[test_index]

    train_y  = y[train_index]
    test_y   = y[test_index]

    model = create_model(snv_data,mRNA_data,miRNA_data)
    model.fit( {"snv_inputs": snv_train_x,  "mRNA_inputs": mRNA_train_x, 'miRNA_inputs':miRNA_train_x},train_y,
                 validation_data=({"snv_inputs": snv_test_x, "mRNA_inputs": mRNA_test_x,'miRNA_inputs':miRNA_test_x},test_y),
                 epochs=150,batch_size = 64,class_weight = {0:x_0,1:x_1})  

    y_pred = model.predict({"snv_inputs": snv_test_x, "mRNA_inputs": mRNA_test_x,'miRNA_inputs':miRNA_test_x})

    y_score = [1 if index>=0.5  else 0 for index in  y_pred]

    evaluate_epoch = get_metrics(test_y,y_score,y_pred)
    print(evaluate_epoch)
    get_weigjts(model,p)
    p = p + 1
    kfscore.append(evaluate_epoch)
    
results = list(np.array(kfscore).sum(axis= 0)/5.0)
print('Cross validated results :  ACC = {}, REC = {}, F1 = {}, AUC = {}, AUPR ={}'.format(results[1],results[2],results[3],results[4],results[5]))

K.permute_dimensions(WK.shape (64, None)
QK.shape (64, 64)
(None, 64)
K.permute_dimensions(WK.shape (64, None)
QK.shape (64, 64)
(None, 64)
K.permute_dimensions(WK.shape (64, None)
QK.shape (64, 64)
(None, 64)
Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 snv_inputs (InputLayer)        [(None, 1000)]       0           []                               
                                                                                                  
 mRNA_inputs (InputLayer)       [(None, 1000)]       0           []                               
                                                                                                  
 miRNA_inputs (InputLayer)      [(None, 200)]        0           []                               
                                                                               

D:\Anaconda3\lib\site-packages\keras\engine\training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


283/283 [==============================] - 1s 4ms/sample - loss: 1.4839 - acc: 0.4806 - val_loss: 1.4687 - val_acc: 0.4930
Epoch 2/150
283/283 [==============================] - 0s 310us/sample - loss: 1.4604 - acc: 0.4806 - val_loss: 1.4455 - val_acc: 0.4930
Epoch 3/150
283/283 [==============================] - 0s 271us/sample - loss: 1.4373 - acc: 0.4806 - val_loss: 1.4228 - val_acc: 0.4930
Epoch 4/150
283/283 [==============================] - 0s 268us/sample - loss: 1.4148 - acc: 0.4806 - val_loss: 1.4007 - val_acc: 0.4930
Epoch 5/150
283/283 [==============================] - 0s 275us/sample - loss: 1.3929 - acc: 0.4806 - val_loss: 1.3792 - val_acc: 0.4930
Epoch 6/150
283/283 [==============================] - 0s 280us/sample - loss: 1.3715 - acc: 0.4806 - val_loss: 1.3583 - val_acc: 0.4930
Epoch 7/150
283/283 [==============================] - 0s 247us/sample - loss: 1.3507 - acc: 0.4806 - val_loss: 1.3379 - val_acc: 0.4930
Epoch 8/150
283/283 [==============================] - 

283/283 [==============================] - 0s 254us/sample - loss: 0.1591 - acc: 0.9929 - val_loss: 0.4342 - val_acc: 0.8169
Epoch 120/150
283/283 [==============================] - 0s 236us/sample - loss: 0.1570 - acc: 0.9929 - val_loss: 0.4333 - val_acc: 0.8169
Epoch 121/150
283/283 [==============================] - 0s 243us/sample - loss: 0.1550 - acc: 0.9929 - val_loss: 0.4323 - val_acc: 0.8169
Epoch 122/150
283/283 [==============================] - 0s 264us/sample - loss: 0.1540 - acc: 0.9929 - val_loss: 0.4320 - val_acc: 0.8169
Epoch 123/150
283/283 [==============================] - 0s 268us/sample - loss: 0.1505 - acc: 0.9929 - val_loss: 0.4298 - val_acc: 0.8310
Epoch 124/150
283/283 [==============================] - 0s 243us/sample - loss: 0.1494 - acc: 0.9965 - val_loss: 0.4285 - val_acc: 0.8310
Epoch 125/150
283/283 [==============================] - 0s 253us/sample - loss: 0.1482 - acc: 0.9965 - val_loss: 0.4274 - val_acc: 0.8310
Epoch 126/150
283/283 [==================

D:\Anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


...

284/284 [==============================] - 0s 263us/sample - loss: 0.1645 - acc: 0.9930 - val_loss: 0.3000 - val_acc: 0.9286
Epoch 120/150
284/284 [==============================] - 0s 267us/sample - loss: 0.1627 - acc: 0.9930 - val_loss: 0.2968 - val_acc: 0.9286
Epoch 121/150
284/284 [==============================] - 0s 284us/sample - loss: 0.1598 - acc: 0.9965 - val_loss: 0.2936 - val_acc: 0.9429
Epoch 122/150
284/284 [==============================] - 0s 272us/sample - loss: 0.1585 - acc: 0.9930 - val_loss: 0.2913 - val_acc: 0.9429
Epoch 123/150
284/284 [==============================] - 0s 284us/sample - loss: 0.1561 - acc: 0.9930 - val_loss: 0.2900 - val_acc: 0.9286
Epoch 124/150
284/284 [==============================] - 0s 376us/sample - loss: 0.1537 - acc: 0.9965 - val_loss: 0.2909 - val_acc: 0.9286
Epoch 125/150
284/284 [==============================] - 0s 270us/sample - loss: 0.1533 - acc: 0.9894 - val_loss: 0.2894 - val_acc: 0.9286
Epoch 126/150
284/284 [==================

D:\Anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


[[33  3]
 [ 2 32]]
None
(0.9142857142857143, 0.9285714285714286, 0.9411764705882353, 0.9275362318840579, 0.9812091503267975, 0.9803075419514934, 0.9800056765697639)


D:\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Cross validated results :  ACC = 0.8843058350100603, REC = 0.8835294117647059, F1 = 0.8801392628082814, AUC = 0.9562024629671686, AUPR =0.9579800600852015


In [34]:
#feature importance
#The average results of the five-fold crossover experiments

import pandas as pd
dirs_paths = './coef_weight/LIHC'
dirs_genes = ['snv.csv','mRNA.csv','miRNA.csv']
dirs = ['h0','h1','h2','h3','h4']

for i in range(0,3):
    t_gene_data = pd.DataFrame()
    for j in dirs:
        print(dirs_paths+'\\'+j+'\\'+dirs_genes[i])
        genes_data = pd.read_csv(dirs_paths+'\\'+j+'\\'+dirs_genes[i])
        t_gene_data = t_gene_data.append(genes_data)
    temp_gene = t_gene_data.groupby('genes').mean().sort_values('values',ascending=False)
    temp_gene.to_csv(dirs_paths+'\\average\\average_'+dirs_genes[i])

./coef_weight/LIHC\h0\snv.csv
./coef_weight/LIHC\h1\snv.csv
./coef_weight/LIHC\h2\snv.csv
./coef_weight/LIHC\h3\snv.csv
./coef_weight/LIHC\h4\snv.csv
./coef_weight/LIHC\h0\mRNA.csv
./coef_weight/LIHC\h1\mRNA.csv
./coef_weight/LIHC\h2\mRNA.csv
./coef_weight/LIHC\h3\mRNA.csv
./coef_weight/LIHC\h4\mRNA.csv
./coef_weight/LIHC\h0\miRNA.csv
./coef_weight/LIHC\h1\miRNA.csv
./coef_weight/LIHC\h2\miRNA.csv
./coef_weight/LIHC\h3\miRNA.csv
./coef_weight/LIHC\h4\miRNA.csv


In [24]:

def create_single_model(inputs,omics_id):
   
    #omics_id   0 
    
    S_inputs = Input(shape=(inputs.shape[1],), dtype='float32')
    


    h0 = Biological_module(gene_pathway_bp_dfs[omics_id].shape[0],mapp =gene_pathway_bp_dfs[omics_id].values.T,W_regularizer=l2(0.001))(S_inputs)
    

    atten1 = Self_Attention(64,W_regularizer=l2(0.001))(h0)
    
    h4 = tf.keras.layers.Dense(32,activation='tanh')(atten1)
    
    h5 = tf.keras.layers.Dense(1,activation='sigmoid')(h4)

    model = Model(inputs=S_inputs, outputs=h5)

    model.summary()

    opt = tf.keras.optimizers.Adam(learning_rate = 0.0001,decay=0.0001) #,decay=-0.0001
    model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=['acc'])
    return model